In [ ]:
%load_ext autoreload
%autoreload 2
import torch

from plstm.torch.plstm_2d import (
    transition_matrices_2d,
    source_matrices_2d,
    mark_matrices_2d,
    gating_matrices_2d,
    pLSTM2D_fwbw,
)
import matplotlib.pyplot as plt

In [ ]:
# X, Y = 25, 24
X, Y = 32, 32

In [ ]:
alpha = -0.00
phi1 = 0.8
gammaX1 = phi1
gammaY1 = 1 - phi1
phi2 = 0.2
phi3 = 0.8
gammaX2 = phi2
gammaY2 = 1 - phi2
gammaX3 = phi3
gammaY3 = 1 - phi3


S0_r = torch.cat(
    [
        gammaX1 * torch.ones([1, X // 2, Y, 1, 1, 1]),
        torch.cat(
            [
                gammaX2 * torch.ones([1, X - X // 2, Y // 2, 1, 1, 1]),
                gammaX3 * torch.ones([1, X - X // 2, Y - Y // 2, 1, 1, 1]),
            ],
            dim=2,
        ),
    ],
    dim=1,
)  # phi*sqrt(phi*(1-phi))*torch.ones([1, X, Y, 1, 1, 1]) + alpha,
S0_d = torch.cat(
    [
        gammaY1 * torch.ones([1, X // 2, Y, 1, 1, 1]),
        torch.cat(
            [
                gammaY2 * torch.ones([1, X - X // 2, Y // 2, 1, 1, 1]),
                gammaY3 * torch.ones([1, X - X // 2, Y - Y // 2, 1, 1, 1]),
            ],
            dim=2,
        ),
    ],
    dim=1,
)

M0_l = torch.ones([1, X, Y, 1, 1, 1]) + alpha
M0_u = torch.ones([1, X, Y, 1, 1, 1]) + alpha

T0_rl = torch.cat(
    [
        gammaX1 * torch.ones([1, X // 2, Y, 1, 1]),
        torch.cat(
            [
                gammaX2 * torch.ones([1, X - X // 2, Y // 2, 1, 1]),
                gammaX3 * torch.ones([1, X - X // 2, Y - Y // 2, 1, 1]),
            ],
            dim=2,
        ),
    ],
    dim=1,
)
T0_du = torch.cat(
    [
        gammaY1 * torch.ones([1, X // 2, Y, 1, 1]),
        torch.cat(
            [
                gammaY2 * torch.ones([1, X - X // 2, Y // 2, 1, 1]),
                gammaY3 * torch.ones([1, X - X // 2, Y - Y // 2, 1, 1]),
            ],
            dim=2,
        ),
    ],
    dim=1,
)
T0_dl = torch.cat(
    [
        gammaY1 * torch.ones([1, X // 2, Y, 1, 1]),
        torch.cat(
            [
                gammaY2 * torch.ones([1, X - X // 2, Y // 2, 1, 1]),
                gammaY3 * torch.ones([1, X - X // 2, Y - Y // 2, 1, 1]),
            ],
            dim=2,
        ),
    ],
    dim=1,
)
T0_ru = torch.cat(
    [
        gammaX1 * torch.ones([1, X // 2, Y, 1, 1]),
        torch.cat(
            [
                gammaX2 * torch.ones([1, X - X // 2, Y // 2, 1, 1]),
                gammaX3 * torch.ones([1, X - X // 2, Y - Y // 2, 1, 1]),
            ],
            dim=2,
        ),
    ],
    dim=1,
)

D0 = torch.zeros([1, X, Y, 1, 1, 1, 1])

In [ ]:
levels = 6
T = transition_matrices_2d(T0_rl, T0_du, T0_dl, T0_ru, levels=levels)
[t[-1].shape for t in T]

In [ ]:
T_rl, T_du, T_dl, T_ru = transition_matrices_2d(T0_rl, T0_du, T0_dl, T0_ru, levels=levels)
S_r, S_d = source_matrices_2d(S0_r, S0_d, T_rl, T_du, T_dl, T_ru, levels=levels)
M_l, M_u = mark_matrices_2d(M0_l, M0_u, T_rl, T_du, T_dl, T_ru, levels=levels)
G = gating_matrices_2d(S_r, S_d, M_l, M_u, T_rl, T_du, T_dl, T_ru, levels=levels)

### Visual Inspection

In [ ]:
plt.imshow((G.transpose(3, 5)[0, 0, :, :, 0, 5, 0, 0, 0, 0] + 1e-5).transpose(0, 1))
plt.colorbar()

In [ ]:
plt.imshow((G.transpose(3, 5)[0, 0, :, :, 0, 0, 7, 0, 0, 0] + 1e-5).transpose(0, 1))
plt.colorbar()

In [ ]:
plt.imshow((G.transpose(3, 5)[0, 0, :, :, 0, 7, 7, 0, 0, 0] + 1e-5).transpose(0, 1))
plt.colorbar()

In [ ]:
plt.imshow((G.transpose(3, 5)[0, 0, :, :, 0, 0, 0, 0, 0, 0] + 1e-5).transpose(0, 1))
plt.colorbar()

### Exemplary Analysis

$ Sr[0,0]*Tld[1,0]*Tud[1,1]*Du[1,2] + Sd[0,0]*(Tud[0,1]*Tur[0,2]*Dl[1,2] + Tur[0,1]*Tld[1,1]*Du[1,2])  $

$ Sr * Tud + Sd * Tud * (X-Tud) + Sd * (X-Tud) 

### Test stmg gradient

In [ ]:
from plstm.torch.plstm_2d import stmg_matrices

In [ ]:
B = 1
X = 8
Y = 8
NX = 2
NY = 2
BP = 4
DHQK = 1
DHHV = 1
JQ = 1
JT = 1
JV = 1
JK = 1
JO = 1
levels = 2

phi0 = 0.6
S0_r = phi0 * torch.ones([B, X, Y, JT, JK, JV])
S0_d = phi1 * torch.ones([B, X, Y, JT, JK, JV])

M0_l = torch.ones([B, X, Y, JO, JQ, JT])
M0_u = torch.ones([B, X, Y, JO, JQ, JT])
D0 = torch.ones([B, X, Y, JO, JQ, JK, JV])

T00 = torch.eye(JT)[None, None, None, :, :] * torch.ones([B, X, Y, 1, 1])
T0_rl = phi0 * T00 + 0.01 * torch.randn_like(T00)
T0_du = phi1 * T00 + 0.01 * torch.randn_like(T00)
T0_dl = phi1 * T00 + 0.01 * torch.randn_like(T00)
T0_ru = phi0 * T00 + 0.01 * torch.randn_like(T00)

for a in [
    S0_r,
    S0_d,
    T0_rl,
    T0_du,
    T0_dl,
    T0_ru,
    M0_l,
    M0_u,
]:
    a.requires_grad_(True)

In [ ]:
D = D0.reshape(-1, NX, BP, 1, NY, BP, 1, JO, JQ, JK, JV)
S_r, S_d, T_rl, T_du, T_dl, T_ru, M_l, M_u, G_nodirect = stmg_matrices(
    S0_r,
    S0_d,
    T0_rl,
    T0_du,
    T0_dl,
    T0_ru,
    M0_l,
    M0_u,
    levels=levels,
)
G = G_nodirect + (
    D
    * torch.eye(BP)[None, None, :, :, None, None, None, None, None, None, None]
    * torch.eye(BP)[None, None, None, None, None, :, :, None, None, None, None]
)

In [ ]:
(
    G.mean() + S_r.mean() + T_rl.mean() + T_dl.mean() + T_ru.mean() + T_du.mean() + S_d.mean() + M_l.mean() + M_u.mean()
).backward()

torch.any(torch.isnan(S0_r.grad))

# pLSTM2D chunkwise

In [ ]:
from plstm.torch.util import log2
from plstm.torch.plstm_2d import pLSTM2D_torch

B = 1
X = 14
Y = 14
BT = 4
DHQK = 1
DHHV = 1
JQ = 1
JT = 1
JV = 1
JK = 1
JO = 1

phi0 = 0.7
phi1 = 1 - phi0

Q = torch.ones([B, X, Y, DHQK, JQ]) / DHQK
K = torch.ones([B, X, Y, DHQK, JK])
V = torch.zeros([B, X, Y, DHHV, JV])
V[0, 0, 0, :, :] = 1.0
S0_r = phi0 * torch.ones([B, X, Y, JT, JK, JV])
S0_d = phi1 * torch.ones([B, X, Y, JT, JK, JV])

M0_l = torch.ones([B, X, Y, JO, JQ, JT])
M0_u = torch.ones([B, X, Y, JO, JQ, JT])
D0 = torch.ones([B, X, Y, JO, JQ, JK, JV])

T00 = torch.eye(JT)[None, None, None, :, :] * torch.ones([B, X, Y, 1, 1])
T0_rl = phi0 * T00 + 0.01 * torch.randn_like(T00)
T0_du = phi1 * T00 + 0.01 * torch.randn_like(T00)
T0_dl = phi1 * T00 + 0.01 * torch.randn_like(T00)
T0_ru = phi0 * T00 + 0.01 * torch.randn_like(T00)
Q.requires_grad_(True)
K.requires_grad_(True)
V.requires_grad_(True)

for X in [S0_r, S0_d, M0_l, M0_u, T0_rl, T0_du, T0_dl, T0_ru, D0]:
    X.requires_grad_(True)


H = pLSTM2D_fwbw(Q, K, V, S0_r, S0_d, T0_rl, T0_du, T0_dl, T0_ru, M0_l, M0_u, D0, levels=log2(BT))

In [ ]:
H.shape

In [ ]:
plt.imshow(H[0, :, :, 0].transpose(0, 1).detach().numpy())

In [ ]:
S0_r.requires_grad

In [ ]:
H.sum().backward()

In [ ]:
G.numel()

## Test gradient

In [ ]:
from plstm.torch.plstm_2d import pLSTM2D_fwbw
from plstm.torch.test_utils import check_forward, check_backward

In [ ]:
# B = 1
# X = 8
# Y = 8
# BT = 4
# DHQK = 1
# DHHV = 2
# JQ = 1
# JT = 1
# JV = 1
# JK = 1
# JO = 1

B = 1
X = 16
Y = 16
BT = 4
DHQK = 1
DHHV = 2
JQ = 2
JT = 7
JV = 3
JK = 5
JO = 4


phi0 = 0.6
phi1 = 1 - phi0

rand_factor = 0.1

Q = 1.0 + rand_factor * torch.randn([B, X, Y, DHQK, JQ]) / DHQK
K = 1.0 + rand_factor * torch.randn([B, X, Y, DHQK, JK])
V = rand_factor * torch.randn([B, X, Y, DHHV, JV]) + torch.arange(JV * DHHV).reshape(1, 1, 1, DHHV, JV)
S0_r = phi0 * (1.0 + rand_factor * torch.randn([B, X, Y, JT, JK, JV]))
S0_d = phi1 * (1.0 + rand_factor * torch.randn([B, X, Y, JT, JK, JV]))
T00 = torch.eye(JT)[None, None, None, :, :] * torch.ones([B, X, Y, 1, 1])
T0_rl = phi0 * T00 + rand_factor * torch.randn_like(T00)
T0_du = phi1 * T00 + rand_factor * torch.randn_like(T00)
# T0_rl = T0_du
T0_dl = phi1 * T00 + rand_factor * torch.randn_like(T00)
T0_ru = phi0 * T00 + rand_factor * torch.randn_like(T00)
M0_l = 1.0 + rand_factor * torch.randn([B, X, Y, JO, JQ, JT])
M0_u = 1.0 + rand_factor * torch.randn([B, X, Y, JO, JQ, JT])
D0 = 1 + rand_factor * torch.randn([B, X, Y, JO, JQ, JK, JV])
Sm = 0.1 - 0.01 * torch.arange(B * X * Y).reshape([B, X, Y]) + 0.2 * torch.randn([B, X, Y])

In [ ]:
def normalize(x, dim=0):
    y = x - torch.mean(x, dim=dim, keepdim=True)
    return y / torch.sqrt(torch.mean(y**2, dim=dim, keepdim=True))

In [ ]:
check_forward(
    lambda q, k, v, s0r, s0d, t0rl, t0du, t0dl, t0ru, m0l, m0u, d0, sm: normalize(
        pLSTM2D_torch(
            q,
            k,
            v,
            s0r,
            s0d,
            t0rl,
            t0du,
            t0dl,
            t0ru,
            m0l,
            m0u,
            d0,
            sm,
            levels=log2(BT) // 2,
        ),
        dim=3,
    ).flatten(3, 4),
    lambda q, k, v, s0r, s0d, t0rl, t0du, t0dl, t0ru, m0l, m0u, d0, sm: normalize(
        pLSTM2D_torch(q, k, v, s0r, s0d, t0rl, t0du, t0dl, t0ru, m0l, m0u, d0, sm, levels=log2(BT)),
        dim=3,
    ).flatten(3, 4),
    (Q, K, V, S0_r, S0_d, T0_rl, T0_du, T0_dl, T0_ru, M0_l, M0_u, D0, Sm),
    verbose=True,
)

In [ ]:
recompute_C, recompute_G = False, True

check_backward(
    lambda q, k, v, s0r, s0d, t0rl, t0du, t0dl, t0ru, m0l, m0u, d0, sm: normalize(
        pLSTM2D_torch(
            q,
            k,
            v,
            s0r,
            s0d,
            t0rl,
            t0du,
            t0dl,
            t0ru,
            m0l,
            m0u,
            d0,
            sm,
            levels=log2(BT) * 2,
        ),
        dim=3,
    ).flatten(3, 4),
    lambda q, k, v, s0r, s0d, t0rl, t0du, t0dl, t0ru, m0l, m0u, d0, sm: normalize(
        pLSTM2D_fwbw(
            q,
            k,
            v,
            s0r,
            s0d,
            t0rl,
            t0du,
            t0dl,
            t0ru,
            m0l,
            m0u,
            d0,
            sm,
            levels=log2(BT),
            recompute_C=recompute_C,
            recompute_G=recompute_G,
        ),
        dim=3,
    ).flatten(3, 4),
    (Q, K, V, S0_r, S0_d, T0_rl, T0_du, T0_dl, T0_ru, M0_l, M0_u, D0, Sm),
    verbose=True,
    atol=0.01,
    rtol=0.05,
)

# Full Layer

In [ ]:
from plstm.torch.plstm_2d_layer import pLSTM2DLayer, pLSTM2DLayerConfig

In [ ]:
B = 2
D = 32
H = 3
X = Y = 64
JK = JQ = 1
JV = JO = JT = 1

plstm = pLSTM2DLayer(
    pLSTM2DLayerConfig(
        mode="P",
        DK=D // JK,
        num_heads=H,
        DV=D // JV,
        JQ=JQ,
        JK=JK,
        JV=JV,
        JT=JT,
        JO=JO,
        input_dim=D,
        sub_heads=8,
        additional_magnitude=False,
    )
)

In [ ]:
x = torch.ones([B, X, Y, H * D])

plstm_tc = torch.compile(plstm)

r = plstm(x)

In [ ]:
r2 = plstm(x)

In [ ]:
r3 = plstm_tc(x)

In [ ]:
r4 = plstm_tc(x)